# Part 1: Spam or ham? (45 marks)

In the spam literature, an email that is **not** spam is called _ham_. 

Your task is to develop a Naive Bayes classifier to determine whether a given email is spam or ham.

## The data: Spam!

The following cell loads your training data set (called _training set_).

In [1]:
import numpy as np

training_spam = np.loadtxt(open("data/training_spam.csv"), delimiter=",")
print("Shape of the spam training data set:", training_spam.shape)
print(training_spam)

Shape of the spam training data set: (1000, 55)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [1. 1. 1. ... 1. 1. 0.]
 [1. 0. 0. ... 1. 1. 1.]]


Your training set consists of 1000 rows and 55 columns. Each row corresponds to one email message. The first column is the _response_ variable and describes whether a message is spam (1) or ham (0). The remaining 54 columns are _features_ that you will use to build a classifier. These features correspond to 54 different keywords (such as "money", "free", and "receive") and special characters (such as ":", "!", and "$"). A feature has the value "1" if the keyword appears in the message and "0" otherwise.

## The model:  Naïve Bayes
Your [naïve Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) classifier will distinguish between two classes:

* **$C = 1$ for spam messages **
* **$C = 0$ for ham messages. **


The classifier builds a model for the probability $P(C=c\ |\ message)$ that a given message belongs to a certain class. A new message is then classified based on the Bayesian *maximum a posteriori* estimate

\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{argmax} \  P(C=c\ |\ message).
\end{equation}

Using Bayes' rule we can write

\begin{equation}
P(C=c\ |\ message) = \frac{P(message\ |\ C=c)P(C=c)}{P(message\ |\ C=1)P(C=1) + P(message\ |\ C=0)P(C=0)}.  \quad \quad 
\end{equation}

The denominator is the same for both classes and we can thus drop it to get

\begin{equation}
P(C=c\ |\ message) \propto P(message\ |\ C=c)P(C=c).
\end{equation}


We represent a message using a binary [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model) model. Specifically, a message is represented as $\mathbf{w} = (w_1, ..., w_k)$, where $w_i = 1$ if the word $w_i$ appears in the message and $w_i = 0$ otherwise. We assume **class-conditional independence between occurences of known words** and can therefore write 

\begin{equation}
P(message\ |\ C=c) = \prod_{i = 1}^k P(w_i\ |\ C=c).
\end{equation}

The classifier now can be written as follows :

\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{argmax} \ [ P(C=c)   \prod_{i = 1}^k P(w_i\ |\ C=c) ].
\end{equation}


### Multinomial Naïve Bayes

Different naïve Bayes models differ in their distributional assumptions of $P(w\ |\ C=c)$, that is, the conditional likelihood of a word $w$ given a class $c$. We will model $P(w\ |\ C=c)$ using a [multinomial distribution](https://en.wikipedia.org/wiki/Multinomial_distribution). Intuitively, the multinomial distribution assumes that the words of a message are "drawn" independently from a bag of $k$ different keywords. Depending on the class membership $c$, each keyword has a probability $\theta_{class, word}$ of being drawn. For example,

* $\theta_{spam, w}$ will have high value for $w \in \{$bank, transfer, buy, viagra... $\}$.
* $\theta_{ham, w}$ will have high value for $w \in \{$paper, conference, proposal, experiment... $\}$, if the training data was mostly gathered from emails of researchers.

Both the class priors $P(C=c)$ and the class-conditional likelihoods $\theta_{c, w} = P(w\ |\ C=c)$ have to be estimated from the training data. The parameters $\theta_{c, w}$ are estimated by counting the relative frequencies in the training data. Use **Laplace-smoothing** with $\alpha = 1$, that is,

\begin{equation}
\theta_{c, w} = \frac{n_{c, w} + \alpha}{n_{c} + k \alpha},
\end{equation}

where $n_{c, w}$ is the number of times the word $w$ appears in messages of class $c$ in the training set, $n_{c}$ is the total count of words for all messages of class $c$, and $k$ is the number of features (key-words).

The likelihood of observing $\mathbf{w}$ in a message of class $c$ is proportional to
\begin{equation}
P(\mathbf{w}\ |\ C=c) \propto \prod_{i = 1}^k  (\theta_{c, i})^{w_i}.
\end{equation}

### Increasing numerical stability
We can increase the numerical stability of the algorithm by taking logarithms of the posterior distributions, that is,
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{argmax} \ log( P(C=c)   \prod_{i = 1}^k P(w_i\ |\ C=c) ) \\
 = \underset{c \in \{0,1\}}{argmax} \ [ log( P(C=c)) + \sum_{i = 1}^k w_i \ log(\theta_{c, i}) ].
\end{equation}

You will implement this more stable version of the algorithm in developing your classifier. 

## Part 1A: Estimate class priors (10 marks)

Define a function called `estimate_log_class_priors()` that takes as input a data set with binary response variable (0s and 1s) in the left-most column and returns a numpy array containing the **the logarithm** of the empirical class priors $P(C=c)$ for $c \in \{0, 1\}$.

In [29]:
def estimate_log_class_priors(data):
    """
    Given a data set with binary response variable (0s and 1s) in the
    left-most column, calculate the logarithm of the empirical class priors,
    that is, the logarithm of the proportions of 0s and 1s:
    log(P(C=0)) and log(P(C=1))

    :param data: a two-dimensional numpy-array with shape = [n_samples, 1 + n_features]
                 the first column contains the binary response (coded as 0s and 1s).

    :return log_class_priors: a numpy array of length two
    """
    ### YOUR CODE HERE...
    
    leftmost = data[:,0]
        
    zeroes = 0
    for i in leftmost:
        if i == 0: zeroes += 1
            
    ones = len(leftmost) - zeroes
    
    priors = np.array([np.log(zeroes/len(leftmost)), np.log(ones/len(leftmost))])
    
    print(np.exp(priors))
        
    return priors

In [30]:
# This is a test cell. Do not delete or change. 
# You can use this cell to check whether the returned objects of your function are of the right data type.
log_class_priors = estimate_log_class_priors(training_spam)
print("result", log_class_priors)

# Check length
assert(len(log_class_priors) == 2)

# Check whether the returned object is a numpy.ndarray
assert(isinstance(log_class_priors, np.ndarray))

# Check wehther the values of this numpy.array are floats.
assert(log_class_priors.dtype == float)


[0.387 0.613]
result [-0.94933059 -0.48939034]


## Part 1B: Estimate class-conditional likelihoods (10 marks)
Define a function called `estimate_log_class_conditional_likelihoods()` that takes as input a data set with binary response variable (0s and 1s) in the left-most column and returns **the logarithm** of the empirical class-conditional likelihoods $log(P(w_i | c_j))$ for all words $w_i$ and both classes ($j \in {0, 1}$). These parameters should be returned in a two-dimensional numpy-array with shape = `[num_classes, num_features]`.

Assume a multinomial feature distribution and use Laplace smoothing with $\alpha = 1$. 

Hint: many `numpy`-functions contain an `axis` argument. If you specify `axis=0`, you can perform column-wise (that is, feature-wise!) computations.

In [65]:
#### not done u idiot

def get_word_count(data):
    
    # column determining if c = 0 or 1
    leftmost = data[:,0]
    
    data = data[:, 1:]
    
    spam_words = 0
    ham_words = 0
    
    
    # add up total word count for all messages of each class
    for i in range(0,data.shape[0]):
        if leftmost[i] == 1:
            spam_words += np.count_nonzero(data[i,:] == 1)
        else:
            ham_words += np.count_nonzero(data[i,:] == 1)    
            
            
            
    return [spam_words,ham_words]

def get_ones(d):
    ones = 0
    for i in d: 
        if i == 1: ones += 1
    
    return ones

def estimate_log_class_conditional_likelihoods(data, alpha=1.0):
    """
    Given a data set with binary response variable (0s and 1s) in the
    left-most column and binary features, calculate the empirical
    class-conditional likelihoods, that is,
    log(P(w_i | c_j)) for all features i and both classes (j in {0, 1}).

    Assume a multinomial feature distribution and use Laplace smoothing
    if alpha > 0.

    :param data: a two-dimensional numpy-array with shape = [n_samples, n_features]

    :return theta:
        a numpy array of shape = [2, n_features]. theta[j, i] corresponds to the
        logarithm of the probability of feature i appearing in a sample belonging 
        to class j.
    """
    
    # defines theta
    theta = np.zeros([2,data.shape[1] - 1])
    
    # gets total number of words for e
    [spam_words, ham_words] = get_word_count(data)
    print("spam_words =", spam_words)
    print("ham_words =",ham_words)
    
    log_class_priors = estimate_log_class_priors(training_spam)
    log_class_priors[0] = np.exp(log_class_priors[0])
    log_class_priors[1] = np.exp(log_class_priors[1])
    
    leftmost = data[:,0]
    
    features = data.shape[1] - 1
    
    # 0 = rows, 1 = columns (rows, columns)
    print("data.shape[1] =", data.shape[1])
    
    # for every column (bar the first one) ...
    for j in range(1, data.shape[1]):
        spam_ncw = 0
        ham_ncw = 0
        
        # for every row in the column...
        for i in range(1, data.shape[0]):
            #... if word does appear, then find if it was spam/ham, then add to right accumulator
            if data[i,j] == 1:
                if leftmost[i] == 1:
                    spam_ncw += 1
                else:
                    ham_ncw += 1   
        
        print("spam_ncw =",spam_ncw)
        print("ham_ncw =",ham_ncw)
                            
        
        # log(p(w | spam)) = log(p(w,spam)/p(spam))
        prob_spam = np.log((spam_ncw + alpha)/(spam_words + alpha * features))
        prob_ham = np.log((ham_ncw + alpha)/(ham_words + alpha*features))
        
        theta[0,j-1] = prob_ham
        theta[1,j-1] = prob_spam
        
    print(theta)
    return theta


In [66]:
# This is a test cell. Do not delete or change. 
# You can use this cell to check whether the returned objects of your function are of the right data type.
log_class_conditional_likelihoods = estimate_log_class_conditional_likelihoods(training_spam, alpha=1.0)

# Check data type(s)
assert(isinstance(log_class_conditional_likelihoods, np.ndarray))

# Check shape of numpy array
assert(log_class_conditional_likelihoods.shape == (2, 54))

# Check data type of array elements
assert(log_class_conditional_likelihoods.dtype == float)


spam_words = 5028
ham_words = 4917
[0.387 0.613]
data.shape[1] = 55
spam_ncw = 146
ham_ncw = 106
spam_ncw = 120
ham_ncw = 60
spam_ncw = 231
ham_ncw = 169
spam_ncw = 6
ham_ncw = 1
spam_ncw = 253
ham_ncw = 139
spam_ncw = 159
ham_ncw = 63
spam_ncw = 157
ham_ncw = 12
spam_ncw = 139
ham_ncw = 40
spam_ncw = 115
ham_ncw = 53
spam_ncw = 183
ham_ncw = 101
spam_ncw = 115
ham_ncw = 37
spam_ncw = 242
ham_ncw = 263
spam_ncw = 124
ham_ncw = 80
spam_ncw = 48
ham_ncw = 27
spam_ncw = 54
ham_ncw = 11
spam_ncw = 220
ham_ncw = 60
spam_ncw = 169
ham_ncw = 64
spam_ncw = 135
ham_ncw = 80
spam_ncw = 346
ham_ncw = 343
spam_ncw = 87
ham_ncw = 12
spam_ncw = 314
ham_ncw = 209
spam_ncw = 18
ham_ncw = 5
spam_ncw = 126
ham_ncw = 16
spam_ncw = 155
ham_ncw = 12
spam_ncw = 11
ham_ncw = 231
spam_ncw = 8
ham_ncw = 171
spam_ncw = 1
ham_ncw = 163
spam_ncw = 5
ham_ncw = 95
spam_ncw = 4
ham_ncw = 80
spam_ncw = 3
ham_ncw = 90
spam_ncw = 1
ham_ncw = 61
spam_ncw = 0
ham_ncw = 47
spam_ncw = 12
ham_ncw = 67
spam_ncw = 0
ham_ncw =

## Part  1C: Classify e-mails (10 marks)

Having calculated the log class priors and the log class-conditional likelihoods for a given training set, define a function called `predict()`that takes a data set of new messages as input and predicts for each message whether it is spam or not. Note that the input should **not** contain a response variable.

Use your `predict()` function to classify the messages of your **training data set** `training_spam`. Compute the accuracy of your algorithm _in the training set_ by comparing your predictions to the true class values. Accuracy is simply defined as the proportion of true predictions made by your classifier. Store the accuracy of your naïve Bayes algorithm in a variable called `training_set_accuracy`.

In [73]:
def predict(new_data, log_class_priors, log_class_conditional_likelihoods):
    """
    Given a new data set with binary features, predict the corresponding
    response for each instance (row) of the new_data set.

    :param new_data: a two-dimensional numpy-array with shape = [n_test_samples, n_features].
    :param log_class_priors: a numpy array of length 2.
    :param log_class_conditional_likelihoods: a numpy array of shape = [2, n_features].
        theta[j, i] corresponds to the logarithm of the probability of feature i appearing
        in a sample belonging to class j.
    :return class_predictions: a numpy array containing the class predictions for each row
        of new_data.
    """
    ### YOUR CODE HERE...
    
    k = log_class_conditional_likelihoods.shape[1]
    print("k=",k)
    
    class_predictions = np.zeros(len(new_data))
    

    prior_ham = log_class_priors[0]
    print("prior_ham =", prior_ham)
    prior_spam = log_class_priors[1]
    print("prior_spam", prior_spam)
        
    # for every value in the "new_data"...
    for i in range(0,len(new_data)):
        acc_spam = 0
        # for every feature in the table...
        for j in range(0,(k)):
            # ... do Wi * log(theta(c,i))
            if new_data[i,j] == 1:
                acc_spam = acc_spam + log_class_conditional_likelihoods[1, j]

        prob_spam = prior_spam + acc_spam

        acc_ham = 0
        for l in range(0,(k)):
            if new_data[i,l] == 1:
                acc_ham = acc_ham + log_class_conditional_likelihoods[0, l]
                
                
        prob_ham = prior_ham + acc_ham

        if prob_ham > prob_spam:
            class_predictions[i] = 0
        else:
            class_predictions[i] = 1
    

    print(new_data)
    print(new_data.shape)
    
    print(class_predictions)
    
    return class_predictions

def accuracy(y_predictions, y_true):
    """
    Calculate the accuracy.
    
    :param y_predictions: a one-dimensional numpy array of predicted classes (0s and 1s).
    :param y_true: a one-dimensional numpy array of true classes (0s and 1s).
    
    :return acc: a float between 0 and 1 
    """
    ### YOUR CODE HERE...
    
    acc = 0
    for i in range(1, len(y_predictions)):
        if y_predictions[i] == y_true[i]: acc += 1
    
    acc = acc/len(y_predictions)
    print(acc * 100)
    
    return acc
    
# training_set_accuracy = ...

In [74]:
# This is a test cell. Do not delete or change. 
# You can use this cell to check whether the returned objects of your function are of the right data type.
class_predictions = predict(training_spam[:, 1:], log_class_priors, log_class_conditional_likelihoods)

# Check data type(s)
assert(isinstance(class_predictions, np.ndarray))

# Check shape of numpy array
assert(class_predictions.shape == (1000,))

# Check data type of array elements
assert(np.all(np.logical_or(class_predictions == 0, class_predictions == 1)))
       
# Check accuracy function
true_classes = training_spam[:, 0]
training_set_accuracy = accuracy(class_predictions, true_classes)
assert(isinstance(training_set_accuracy, float))
assert(0 <= training_set_accuracy <= 1)


k= 54
prior_ham = -0.9493305859523552
prior_spam -0.48939034304592566
[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [1. 1. 1. ... 1. 1. 0.]
 [0. 0. 1. ... 1. 1. 1.]]
(1000, 54)
[1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1.
 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0.
 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1.
 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1.
 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1.
 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1.
 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0.
 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1.

## Part 1D: Classifying previously unseen data (10 marks).
The following cell loads a new set of 500 messages.

In [76]:
testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",")
print("Shape of the testing spam data set:", testing_spam.shape)
testing_spam

Shape of the testing spam data set: (500, 55)


array([[1., 0., 0., ..., 1., 1., 1.],
       [1., 1., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

Use the naïve Bayes algorithm that you trained on `training_spam` in order to classify all messages in the `testing_spam` data set. Store the resulting accuracy in a variable called `testing_set_accuracy`.

In [72]:
class_predictions = predict(testing_spam, log_class_priors, log_class_conditional_likelihoods)
true_classes = testing_spam[:, 0]
testing_set_accuracy = accuracy(class_predictions, true_classes)
print(testing_set_accuracy)

k= 54
prior_ham = -0.9493305859523552
prior_spam -0.48939034304592566
[[1. 0. 0. ... 1. 1. 1.]
 [1. 1. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
(500, 55)
[1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1.
 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.
 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0.
 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1.
 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 

In [10]:
# This is a test cell. Do not delete or change.

## Part 1E: Discussion (5 marks).


In 3 sentences or less: Compare the accuracy of your classifier on the training set and on the test set. Are they different? If yes, how do you explain the difference?

YOUR ANSWER HERE



# Part 2: Zero or One? (55 marks)

In this part of the coursework, you will develop a classifier for a different problem. All you will be given about this problem is a training data set. Your objective is to develop a classifier that will have the highest accuracy in unseen examples.

The following cell loads the training data set.

In [11]:
import numpy as np

training_data = np.loadtxt(open("data/training_data_part_2.csv"), delimiter=",")
print("Shape of the training data set:", training_data.shape)
print(training_data)

Shape of the training data set: (5000, 39)
[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 1. 0.]
 [1. 1. 1. ... 1. 0. 0.]
 ...
 [0. 0. 1. ... 0. 1. 0.]
 [1. 0. 1. ... 0. 1. 0.]
 [1. 1. 0. ... 0. 0. 0.]]


The first column is again the response variable. The remaining 38 columns are binary features. You have multiple tasks:

(1) Your first task is to write a function called `train()` that takes `training_data` as input and returns all the fitted parameters of your model. Note that the fitted parameters of your model depend on the model you choose. For example, if you use a naïve Bayes classifier, you could return a list of class priors and conditional likelihoods. (This function will allow us to compute your model on the fly. We should be able to execute it in less than 10 minutes.) 

(2) Your second task is to provide a variable called `fitted_model` which stores the model parameters you found by executing your train() function on the training_data. If your train function takes more than 20 seconds to run, this variable should load precomputed parameter values (possibly from a file) rather than execute the train() function. 

In [12]:
def train(training_data):
    """
    Train a model on the training_data

    :param training_data: a two-dimensional numpy-array with shape = [200, 12] 
    
    :return fitted_model: any data structure that captures your model
    """
    ### YOUR CODE HERE...
    
    return fitted_model

## Uncomment one of the following two lines depending on whether you want us to compute your model on the 
## fly or load a supplementary file.

# fitted_model = train(training_data)
# fitted_model = load(local_file)

(3) Your third task is to provide a function called `test()` that uses your `fitted_model` to classify the observations of `testing_data`. The `testing_data` is hidden and may contain any number of observations (rows). It contains 38 columns that have the same structure as the features of `training_data`. 

In [13]:
def test(testing_data, fitted_model):
    """
    Classify the rows of testing_data using a fitted_model. 

    :param testing_data: a two-dimensional numpy-array with shape = [n_test_samples, 11]
    :param fitted_model: the output of your train function.

    :return class_predictions: a numpy array containing the class predictions for each row
        of testing_data.
    """
    ### YOUR CODE HERE...
    
    return class_predictions

In [14]:
# This is a test cell. Do not delete or change. 
# You can use this cell to check whether the returned objects of your function are of the right data type.

# Test data types if input are the first 20 rows of the training_data.
class_predictions = test(training_data[:20, 1:], fitted_model)

# Check data type(s)
assert(isinstance(class_predictions, np.ndarray))

# Check shape of numpy array
assert(class_predictions.shape == (20,))

# Check data type of array elements
assert(np.all(np.logical_or(class_predictions == 0, class_predictions == 1)))


NameError: name 'fitted_model' is not defined

Describe in less than 10 sentences: Explain your classifier. Comment on its performance. What other alternative classifiers did you consider or experiment with? How does the performance of your classifier change as the size of the training set increases? You may want to include figures. 

YOUR ANSWER HERE

